In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd Data

/content/drive/MyDrive/Data


In [5]:
ls

 angle_gait.csv              eye_open/           non_PRC.csv     titanic_train.csv
'Clinical Assessment.xlsx'   final_merged.csv    par_logit.csv   weather.csv
 data.csv                    looking_back/       PRC.csv         머신러닝/
 eye_close/                  non_par_logit.csv   stand/


In [6]:
import pandas as pd
non_PRC = pd.read_csv('looking_back/look.csv')
PRC = pd.read_csv('looking_back/look_par.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [8]:
warnings.filterwarnings(action='ignore')

In [9]:
score = clinical_assessment.iloc[:,9]

In [10]:
score = score.dropna()
del score[0]
del score[1]
del score[9]
del score[18]

In [11]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-paretic Side

In [12]:
df_non = pd.DataFrame(non_PRC)

In [13]:
y = pd.DataFrame(y)

In [14]:
non = pd.concat([y,df_non],axis=1)

In [15]:
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34',
       'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41'],
      dtype='object')

In [16]:
X = non.loc[:,['PC4','PC8','PC14','PC16','PC19','PC28']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### logistic regression

In [17]:
lr_model = LogisticRegression()

lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.4
Average Precision: 0.48339999999999994
Average F1-score: 0.5114000000000001
Average AUC: 0.0666


### Random Forest

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7001999999999999
Average Precision: 0.7001999999999999
Average F1-score: 0.8228
Average AUC: 0.5


### SVM Linear

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.45
Average Precision: 0.5833999999999999
Average F1-score: 0.5314
Average AUC: 0.0666


### SVM Poly

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7001999999999999
Average Precision: 0.7001999999999999
Average F1-score: 0.8228
Average AUC: 0.2


### XGBoost

In [21]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7001999999999999
Average Precision: 0.7001999999999999
Average F1-score: 0.8228
Average AUC: 0.4332


## paretic-Side

In [22]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [23]:
import random
random.seed(1)
X = par.loc[:,['PC4','PC7','PC25','PC29','PC31','PC32']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### logistic regression

In [24]:
lr_model = LogisticRegression()

lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.8168
Average Precision: 0.8168
Average F1-score: 0.8914
Average AUC: 0.9


### paretic RandomForest

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7001999999999999
Average Precision: 0.7001999999999999
Average F1-score: 0.8228
Average AUC: 0.9334


###SVM Linear


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7667999999999999
Average Precision: 0.8168
Average F1-score: 0.8513999999999999
Average AUC: 0.7333999999999999


### SVM Poly

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7501999999999999
Average Precision: 0.7501999999999999
Average F1-score: 0.8513999999999999
Average AUC: 0.7


### XGBoost

In [28]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.5],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6502000000000001
Average Precision: 0.7501999999999999
Average F1-score: 0.7514
Average AUC: 0.9
